In [1]:
import pandas as pd

excel_file = pd.ExcelFile('cs_plot.xlsx')
sheet_names = excel_file.sheet_names

# 创建新的Excel文件
with pd.ExcelWriter('kde.xlsx', engine='openpyxl') as writer:
    for sheet_name in sheet_names:
        # 读取每个sheet
        df = pd.read_excel('cs_plot.xlsx', sheet_name=sheet_name)

        # 提取指定行
        target_rows = [148, 440, 731, 1022, 1313, 1604, 1895, 2186, 2477]
        extracted_data = df.iloc[target_rows].copy()

        # 添加原始行号列
        extracted_data.insert(0, 'Original_Row', [row-2 for row in target_rows])

        # 保存到新的sheet
        extracted_data.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"所有sheet的数据已提取并保存到: kde.xlsx")

所有sheet的数据已提取并保存到: kde.xlsx


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from KDEpy import FFTKDE
from KDEpy.bw_selection import silvermans_rule
import warnings
warnings.filterwarnings('ignore')

In [2]:
COLORS = [
    '#0173B2',  # QRLASSOG - 蓝色
    '#DE8F05',  # QRFG - 橙色
    '#029E73',  # QRNN2G - 绿色
    '#CC78BC',  # MCQRNNG - 粉紫色
    '#CA9161',  # VRLG - 棕色
    '#949494',  # WRGG - 灰色
    '#ECA1A6',  # ERMG - 粉红色
    '#56B4E9',  # TSMG - 浅蓝色
    '#D55E00',  # TRQG - 橙红色
    '#8B0000',  # TRMG - 深红色（非黑色的深色）
]

MODEL_NAMES = ['QRLASSOG', 'QRFG', 'QRNN2G', 'MCQRNNG', 'VRLG',
               'WRGG', 'ERMG', 'TSMG', 'TRQG', 'TRMG']

# 科研风格设置
plt.rcParams.update({
    'font.family': 'Arial',
    'font.size': 11,
    'axes.linewidth': 1.0,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'axes.titleweight': 'bold',
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
    'axes.unicode_minus': False,
})

# ============================================================================
# KDE计算函数（使用KDEpy）
# ============================================================================

def calculate_kde(quantile_values, n_points=1000):
    """
    使用KDEpy计算KDE概率密度

    参数:
        quantile_values: 分位数预测值数组
        n_points: KDE评估点数

    返回:
        x, y: KDE曲线的x和y坐标
    """
    # 数据归一化
    pred_min = quantile_values.min()
    pred_max = quantile_values.max()

    if pred_max == pred_min:
        # 如果所有值相同，返回单点分布
        return np.array([pred_min]), np.array([1.0])

    pred_norm = (quantile_values - pred_min) / (pred_max - pred_min)

    # 计算带宽（使用Silverman规则）
    len_data = len(pred_norm)
    if len_data > 1:
        # 计算最优带宽
        tqr = np.percentile(pred_norm, [25, 75])
        IQR = tqr[1] - tqr[0]
        std_val = np.std(pred_norm)

        if std_val > 0 and IQR > 0:
            optimal_bw = 1.06 * min(std_val, IQR/1.34) * (len_data ** (-1/5))
        else:
            optimal_bw = silvermans_rule(pred_norm.reshape(-1, 1))
    else:
        optimal_bw = 0.1

    # 使用FFTKDE计算KDE
    try:
        kde = FFTKDE(bw=optimal_bw, kernel='gaussian')
        x_norm, y = kde.fit(pred_norm).evaluate(n_points)

        # 反归一化
        x = x_norm * (pred_max - pred_min) + pred_min

        return x, y
    except Exception as e:
        print(f"KDE计算错误: {e}")
        return None, None



def plot_kde_with_actual(kde_file='kde_restructured.xlsx',
                        actual_file='actual.xlsx',
                        output_file='kde_plot.png',
                        dpi=300):
    """
    绘制3x3 KDE图，带实际值标记

    参数:
        kde_file: KDE数据文件路径
        actual_file: 实际值数据文件路径
        output_file: 输出PNG文件名
        dpi: 图片分辨率
    """

    print("="*70)
    print("KDE概率密度曲线绘图（使用KDEpy）")
    print("="*70)

    # 读取数据
    print("\n正在读取数据...")
    excel_data = pd.ExcelFile(kde_file)
    sheet_names = excel_data.sheet_names
    actual_df = pd.read_excel(actual_file)
    print(f"✓ 读取 {len(sheet_names)} 个时间点")

    # 创建图表 - 调整布局以容纳外部图例
    fig = plt.figure(figsize=(22, 16), dpi=dpi)

    # 创建网格布局：为图例留出空间
    gs = fig.add_gridspec(3, 3, left=0.06, right=0.85, top=0.94, bottom=0.06,
                          hspace=0.25, wspace=0.25)

    axes = []
    for i in range(3):
        for j in range(3):
            axes.append(fig.add_subplot(gs[i, j]))

    print("\n正在绘制图表...")

    # 为每个时间点绘图
    for idx, sheet_name in enumerate(sheet_names):
        ax = axes[idx]
        df = pd.read_excel(kde_file, sheet_name=sheet_name)
        original_row = df['Original_Row'].iloc[0]

        # 获取对应的实际值
        actual_value = actual_df.iloc[original_row, 0]

        # 为每个模型绘制KDE
        for model_idx, model_name in enumerate(MODEL_NAMES):
            model_data = df[df['Model'] == model_name]

            if model_data.empty:
                continue

            # 提取分位数值
            quantile_cols = [col for col in model_data.columns
                           if col not in ['Model', 'Original_Row']]
            quantile_values = model_data[quantile_cols].values.flatten()
            quantile_values = quantile_values[~np.isnan(quantile_values)]

            if len(quantile_values) < 2:
                continue

            # 使用KDEpy计算KDE
            x, y = calculate_kde(quantile_values, n_points=1000)

            if x is not None and y is not None:
                # TRMG特殊处理：更粗的线条
                if model_name == 'TRMG':
                    linewidth = 3.0
                    alpha = 0.95
                    zorder = 10
                else:
                    linewidth = 2.0
                    alpha = 0.75
                    zorder = 5

                # 只在第一个子图添加label（用于共享图例）
                label = model_name if idx == 0 else None

                ax.plot(x, y,
                       color=COLORS[model_idx],
                       linewidth=linewidth,
                       label=label,
                       alpha=alpha,
                       zorder=zorder)

        # 添加实际值虚线（黑色加粗）
        ax.axvline(actual_value, color='black', linestyle='--',
                  linewidth=2.5, alpha=0.8, zorder=15,
                  label='Actual Value' if idx == 0 else None)

        # 设置子图
        ax.set_xlabel('Offshore Wind Power (MW)', fontsize=11)
        ax.set_ylim(bottom=0)
        ax.set_ylabel('Probability Density', fontsize=11)
        ax.set_title(f'Time Point {original_row}\n(Actual: {actual_value:.1f} MW)',
                    fontsize=12, fontweight='bold', pad=8)
        ax.grid(True, alpha=0.25, linestyle='--', linewidth=0.5)
        ax.tick_params(labelsize=10)

        # 美化边框
        for spine in ax.spines.values():
            spine.set_linewidth(0.8)

        print(f"  [{idx+1}/9] Time Point {original_row} 完成")

    # 创建共享图例（在图外右侧）
    handles, labels = axes[0].get_legend_handles_labels()

    # 在图的右侧添加图例
    fig.legend(handles, labels,
              loc='center left',
              bbox_to_anchor=(0.86, 0.5),
              frameon=True,
              framealpha=0.95,
              edgecolor='black',
              fancybox=True,
              fontsize=11,
              title='Models',
              title_fontsize=12)


    # 保存图片
    print(f"\n正在保存图片...")
    plt.savefig(output_file, dpi=dpi, bbox_inches='tight', format='png')
    print(f"✓ 图片已保存: {output_file}")
    print(f"  分辨率: {dpi} DPI")
    print(f"  格式: PNG")
    print(f"  使用KDE方法: KDEpy FFTKDE")

    plt.close()

    print("\n" + "="*70)
    print("绘图完成!")
    print("="*70)

In [3]:
if __name__ == "__main__":
    # 文件路径（根据实际情况调整）
    kde_file = 'kde_restructured.xlsx'
    actual_file = 'actual.xlsx'

    # 生成图表
    plot_kde_with_actual(
        kde_file=kde_file,
        actual_file=actual_file,
        output_file='../../可视化/kde_plot_kdepy.png',
        dpi=400  # 可调整为600获得更高分辨率
    )

KDE概率密度曲线绘图（使用KDEpy）

正在读取数据...
✓ 读取 9 个时间点

正在绘制图表...
  [1/9] Time Point 146 完成
  [2/9] Time Point 438 完成
  [3/9] Time Point 729 完成
  [4/9] Time Point 1020 完成
  [5/9] Time Point 1311 完成
  [6/9] Time Point 1602 完成
  [7/9] Time Point 1893 完成
  [8/9] Time Point 2184 完成
  [9/9] Time Point 2475 完成

正在保存图片...
✓ 图片已保存: kde_plot_kdepy.png
  分辨率: 400 DPI
  格式: PNG
  使用KDE方法: KDEpy FFTKDE

绘图完成!


In [11]:
import gc
gc.collect()

0